# Projeto 1 - Ciência dos Dados

Nome: Carolina Hirschheimer

Nome: Rafael Evangelista Monteiro

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'KitKat.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo KitKat.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,"@sabeauanyz pra sempre e depois, é você 🤍",0
1,acho que os fabricantes do kitkat colocam coca...,2
2,"rt @jackwrlght: no twitter você não tem fãs, v...",0
3,comi meu último kitkat ontem agora tô morrendo...,2
4,um aí da civil q n cumprimenta e chega falando...,1


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test

,Teste,Relevância
0,rt @jichuwaifu: comprei 2 kitkat\ncomi um e tô...,2
1,@kitkatbrasil bom dia kitkat!!! quando chega e...,2
2,mordi um kitkat de morango pela priemtia vez e...,2
3,meu tio de sp que vai mandar um bolo de kitkat...,2
4,@animalaleatory @larinha_af eu tirando teu ki...,1
...,...,...
495,"@tsoonade não, eu concordo que kitkat e snicke...",2
496,@negohneyy joguei o kitkat por cima como se fo...,2
497,"@barbosa_crf7 tô passando mal, tô passando mal...",0
498,rt @taekook03179038: ninguém sabia se a jisoo ...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto escolhido é o chocolate da marca KitKat. Foram considerados <b>irrelevantes</b> (0) todos os tweets que não referiam-se de forma alguma ao produto. Já os tweets <b>neutros</b> (1) correspodiam àqueles que referiam-se ao KitKat, porém não apontavam nenhuma opinião relevante em relação ao produto. Por fim, os tweets que não apenas referiam-se ao KitKat, mas também apresentavam opiniões sobre ele foram classificados como <b>relevantes</b> (2).

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
train['Relevância'] = train['Relevância'].astype('category')
test['Relevância'] = test['Relevância'].astype('category')

train['Relevância'].cat.categories = ['Irrelevante','Neutro','Relevante']
test['Relevância'].cat.categories = ['Irrelevante','Neutro','Relevante']

In [6]:
test.groupby(by="Relevância").describe()

Teste                                                            \
            count unique                                                top   
Relevância                                                                    
Irrelevante   238    238  hoje o vasco joga amorrr 💢💢💢♥️\n\n- e eu tô co...   
Neutro         91     91  “poste uma foto da sua mesa de trabalho agora”...   
Relevante     171    171  pensando no dia que vou ser recebida com kitka...   

                  
            freq  
Relevância        
Irrelevante    1  
Neutro         1  
Relevante      1

In [7]:
train.groupby(by="Relevância").describe()

Treinamento         \
                  count unique   
Relevância                       
Irrelevante         372    372   
Neutro              135    135   
Relevante           243    243   

                                                                     
                                                           top freq  
Relevância                                                           
Irrelevante                        @jeni_kitkat meu sonho ❤️❤️    1  
Neutro       eu no supermercado aí o meu era só 2 kitkat e ...    1  
Relevante        ando a desejar um mcflury de kitkat há 3 dias    1

In [8]:
# DIVISÃO DO DATAFRAME POR RELEVÂNCIA

filtra_relevantes = train["Relevância"]=="Relevante"
filtra_neutros = train["Relevância"]=="Neutro"
filtra_irrelevantes = train["Relevância"]=="Irrelevante"

relevante = train.loc[filtra_relevantes,:]
neutro = train.loc[filtra_neutros,:]
irrelevante = train.loc[filtra_irrelevantes,:]

In [9]:
# LIMPEZA DOS TWEETS 

import re
import emoji
import functools
import operator

def cleanup(text):
    punctuation = '["\n"!-.:?;()''"",]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
        
# Criação da série limpa de tweets de treinamento

treinamento_string = ""
for i in train.Treinamento:
    treinamento_string+=" "+i+" "
treinamento_string_limpa = cleanup(treinamento_string.lower())
split_emoji = emoji.get_emoji_regexp().split(treinamento_string_limpa)
split_whitespace = [substr.split() for substr in split_emoji]
todas_palavras_treinamento = functools.reduce(operator.concat, split_whitespace)
serie_treinamento = pd.Series(todas_palavras_treinamento)

# Criação da série limpa de tweets relevantes

relevante_string = ""
for i in relevante.Treinamento:
    relevante_string+=" "+i+" "
relevante_string_limpa = cleanup(relevante_string.lower())
split_emoji = emoji.get_emoji_regexp().split(relevante_string_limpa)
split_whitespace = [substr.split() for substr in split_emoji]
todas_palavras_relevante = functools.reduce(operator.concat, split_whitespace)
serie_relevante = pd.Series(todas_palavras_relevante)

# Criação da série limpa de tweets neutros  

neutro_string = ""
for i in neutro.Treinamento:
    neutro_string+=" "+i+" "
neutro_string_limpa = cleanup(neutro_string.lower())
split_emoji = emoji.get_emoji_regexp().split(neutro_string_limpa)
split_whitespace = [substr.split() for substr in split_emoji]
todas_palavras_neutro = functools.reduce(operator.concat, split_whitespace)
serie_neutro = pd.Series(todas_palavras_neutro)

# Criação da série limpa de tweets irrelevantes

irrelevante_string = ""
for i in irrelevante.Treinamento:
    irrelevante_string+=" "+i+" "
irrelevante_string_limpa = cleanup(irrelevante_string.lower())
split_emoji = emoji.get_emoji_regexp().split(irrelevante_string_limpa)
split_whitespace = [substr.split() for substr in split_emoji]
todas_palavras_irrelevante = functools.reduce(operator.concat, split_whitespace)
serie_irrelevante = pd.Series(todas_palavras_irrelevante)

In [10]:
# CRIAÇÃO DAS TABELAS DE FREQUÊNCIA ABSOLUTA:
    
tabela_treinamento_absoluta = serie_treinamento.value_counts() # Série de treinamento
    
tabela_relevante_absoluta = serie_relevante.value_counts() # Série de tweets relevantes

tabela_neutro_absoluta = serie_neutro.value_counts() # Série de tweets neutros

tabela_irrelevante_absoluta = serie_irrelevante.value_counts() # Série de tweets irrelevantes

Aplicando o teorema de Naive-Bayes no caso em questão, obtém-se:

$P(Relevante|tweet) = \frac{P(tweet|Relevante)P(Relevante)}{P(tweet)}$

$P(Neutro|tweet) = \frac{P(tweet|Neutro)P(Neutro)}{P(tweet)}$

$P(Irrelevante|tweet) = \frac{P(tweet|Irrelevante)P(Irrelevante)}{P(tweet)}$

In [15]:
# Cálculo das probabilidades totais
P_relevante = tabela_relevante_absoluta.sum() / tabela_treinamento_absoluta.sum()
P_neutro = tabela_neutro_absoluta.sum() / tabela_treinamento_absoluta.sum()
P_irrelevante = tabela_irrelevante_absoluta.sum() / tabela_treinamento_absoluta.sum()

# Inicializando variáveis para o smoothing:
a = 1
v = 100000

# Criação de lista onde será posteriormente guardada a classificação de cada tweet
classificacao = []

# Implementação do Naive-Bayes:

for tweet in test.Teste:
    
    # Limpa cada tweet na série Teste
    tweet_limpo = cleanup(tweet.lower())
    split_emoji = emoji.get_emoji_regexp().split(tweet)
    split_whitespace = [substr.split() for substr in split_emoji]
    palavras_tweet = functools.reduce(operator.concat, split_whitespace)
    
    # Calcula probabilidade do tweet ser relevante
    P_tweet_dado_relevante = 1 
    for palavra in palavras_tweet:
        if palavra in tabela_relevante_absoluta:
            P_palavra_dado_relevante = (tabela_relevante_absoluta[palavra] + a) / (tabela_relevante_absoluta.sum() + a*v)
        else:
            P_palavra_dado_relevante = (a) / (tabela_relevante_absoluta.sum() + a*v)
        P_tweet_dado_relevante *= P_palavra_dado_relevante
    
    # Calcula probabilidade do tweet ser neutro
    P_tweet_dado_neutro = 1 
    for palavra in palavras_tweet:
        if palavra in tabela_neutro_absoluta:
            P_palavra_dado_neutro = (tabela_neutro_absoluta[palavra] + a) / (tabela_neutro_absoluta.sum() + a*v)
        else:
            P_palavra_dado_neutro = (a) / (tabela_neutro_absoluta.sum() + a*v)
        P_tweet_dado_neutro *= P_palavra_dado_neutro
    
    # Calcula probabilidade do tweet ser irrelevante
    P_tweet_dado_irrelevante = 1 
    for palavra in palavras_tweet:
        if palavra in tabela_irrelevante_absoluta:
            P_palavra_dado_irrelevante = (tabela_irrelevante_absoluta[palavra] + a) / (tabela_irrelevante_absoluta.sum() + a*v)
        else:
            P_palavra_dado_irrelevante = (a) / (tabela_irrelevante_absoluta.sum() + a*v)
        P_tweet_dado_irrelevante *= P_palavra_dado_irrelevante
        
    # Comparação das probabilidades por Naive-Bayes 
    # e classificação em uma nova coluna "Releância por Naive-Bayes, na série Teste"
    P_relevante_dado_tweet_vezes_P_tweet = P_tweet_dado_relevante*P_relevante
    P_neutro_dado_tweet_vezes_P_tweet = P_tweet_dado_neutro*P_neutro
    P_irrelevante_dado_tweet_vezes_P_tweet = P_tweet_dado_irrelevante*P_irrelevante
    
    if P_relevante_dado_tweet_vezes_P_tweet>P_neutro_dado_tweet_vezes_P_tweet and P_relevante_dado_tweet_vezes_P_tweet>P_irrelevante_dado_tweet_vezes_P_tweet:
        classificacao.append("Relevante")
    
    if P_neutro_dado_tweet_vezes_P_tweet>P_relevante_dado_tweet_vezes_P_tweet and P_neutro_dado_tweet_vezes_P_tweet>P_irrelevante_dado_tweet_vezes_P_tweet:
        classificacao.append("Neutro")
        
    if P_irrelevante_dado_tweet_vezes_P_tweet>P_relevante_dado_tweet_vezes_P_tweet and P_irrelevante_dado_tweet_vezes_P_tweet>P_neutro_dado_tweet_vezes_P_tweet:
        classificacao.append("Irrelevante")
        
test['Relevância por Naive Bayes'] = classificacao
test.head(5)

,Teste,Relevância,Relevância por Naive Bayes
0,rt @jichuwaifu: comprei 2 kitkat\ncomi um e tô...,Relevante,Neutro
1,@kitkatbrasil bom dia kitkat!!! quando chega e...,Relevante,Irrelevante
2,mordi um kitkat de morango pela priemtia vez e...,Relevante,Relevante
3,meu tio de sp que vai mandar um bolo de kitkat...,Relevante,Relevante
4,@animalaleatory @larinha_af eu tirando teu ki...,Neutro,Relevante
5,amizade né https://t.co/wn9xrqsy1b,Irrelevante,Irrelevante
6,@h3y_catra eu não ri disso não né,Irrelevante,Irrelevante
7,rt @luddiany: não gosta da any? não me siga!\n...,Irrelevante,Irrelevante
8,rt @deliciasmonique: novidade!! copo de browni...,Neutro,Irrelevante
9,a minha mãe antes de eu pisar em stm: vai gast...,Irrelevante,Irrelevante


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [12]:
tabela = pd.crosstab(test["Relevância"],test["Relevância por Naive Bayes"],normalize="all")
tabela

Relevância por Naive Bayes,Irrelevante,Neutro,Relevante
Relevância,,,
Irrelevante,0.416,0.002,0.058
Neutro,0.066,0.006,0.110
Relevante,0.050,0.008,0.284


In [13]:
P_verdadeiros_positivos = tabela.iloc[2,2]
P_falsos_positivos = tabela.iloc[1,2] + tabela.iloc[0,2]
P_verdadeiros_negativos = tabela.iloc[0,0]
P_falsos_negativos = tabela.iloc[1,0] + tabela.iloc[2,0]

print("Probabilidade de verdadeiros positivos: ",(P_verdadeiros_positivos*100).round(2),"%")
print("Probabilidade de falsos positivos: ",(P_falsos_positivos*100).round(2),"%")
print("Probabilidade de verdadeiros negativos: ",(P_verdadeiros_negativos*100).round(2),"%")
print("Probabilidade de falsos negativos: ",(P_falsos_negativos*100).round(2),"%")

Probabilidade de verdadeiros positivos:  28.4 %
Probabilidade de falsos positivos:  16.8 %
Probabilidade de verdadeiros negativos:  41.6 %
Probabilidade de falsos negativos:  11.6 %


___
### Concluindo

A partir do classificador Naive-Bayes, entre os tweets da série de teste, <b>28,4%</b> foram classificados corretamente como relevantes, enquanto <b>16,8%</b> foram classificados incorretamente como relevantes, ao passo em que <b>41,6%</b> foram classificados corretamente como irrelevantes e <b>11,6%</b> foram classificados incorretamente como irrelevantes.

Tendo isso em vista, pode-se considerar que o classificador construído <b>não possui uma excelente performance</b>. Devido à maior quantidade de dados irrelevantes, relevantes e neutros, respectivamente, pode-se considerar que o classificador teve uma melhor performance, quanto maior o número de tweets analisados. Com isso, explica-se que a taxa de verdadeiros negativos é maior que a de verdadeiros positivos. 

Nesse sentido, é interessante observar como os tweets envolvendo <b>sarcasmo</b> e <b>dupla negação</b> foram interpretados. Como exemplo de dupla negação, pode-se analisar o tweet <i>"@h3y_catra eu não ri disso não né"</i> que foi corretamente classificado pelo Naive Bayes como irrelevante. Além disso, o tweet de dupla negação <i>"odeio kitkat véi bagui muito caro pro tamanho dele e nem é tão bom assim!"</i> também foi classificado corretamente, no caso, como relevante. Enquanto isso, para verificar como tweets com sarcasmo são interpretados, é possível analisar a classificação do seguinte tweet: <i>"@yaaxbb fala não! comi um kitkat só pra aumentar a vontade!"</i>, que foi classificado corretamente como relevante. Dessa forma, conclui-se que os tweets de dupla negação e sarcasmo não geraram grandes impactos de falsos positivos ou negativos. 

Finalmente, gostaria de enfatizar que o classificador, apesar de ainda estar suscetível a melhorias, pode ainda ser expandido para análise de uma série de outros fatores com relação à marca. Se fosse alvo de um <b>plano de expansão</b>, além de dividir os tweets conforme relância, o classificador poderia segmentá-los em tweets que falam sobre o KitKat positivamente ou negativamente, bem como os tweets que tem relação com algum produto específico, por exemplo o lançamento de um novo sabor de KitKat e a avaliação da sua recepção por parte dos consumidores. Portanto, é de grande interesse da própria empresa <b>continuar financiando</b> o projeto em questão, já que ele tem potencial para fornecer diversos dados que podem direcionar medidas da companhia com relação a campanhas de marketing, criação de novos sabores, entre diversos outros aspectos. 

#### <u> Outras reflexões relevantes:  </u>

#### 1) Por que não é possível alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets?

Não é possível alimentar a base de treinamento usando o classificador, uma vez que o objetivo do dataframe de treinamento é justamente ter uma base de dados que com certeza está categorizada corretamente, para que essa base de dados seja utilizada para "ensinar" o classificador quais palavras dão indícios de que o tweet é mais ou menos relevante. Nesse sentido, é crucial que a classificação na série de treinamento seja feita pelo homem, para que o classificador Naive-Bayes "aprenda" de uma base de dados correta. Caso a alimentação fosse realizada pelo classificador, haveria uma margem de tweets classificados incorretamente como relevantes, ou irrelevantes, de modo que isso aumentaria a taxa de erro do classificador, que já aprenderia de uma base de dados que possui falsos positivos e negativos. 

#### 2) Outros cenários de uso para o classificador Naive-Bayes:

O classificador Naive Bayes pode ser utilizado em uma série de outros cenários, além do aqui apresentado. Entre eles, pode-se identificar:

* classificação de emails da caixa de entrada de um usuário como possível spam ou não-spam;
* análise das transações de um cartão de crédito e classificação de possível fraude;
* classificação de pacientes de acordo com a probabilidade de terem determinado diagnóstico dado o laudo de exames;
* classificação de possíveis posts que um usuário gostaria de ver no feed, dados os posts que ele curte atualmente.

#### 3) Melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer. Indique material de pesquisa sobre o assunto).

A fim de melhorar a eficácia do classificador Naive Bayes, seria possível realizar as seguintes melhorias no código:

* <b>Calcular as probabilidades em log.</b> Como as probabilidades são números entre 0 e 1, ao multiplicar as probabilidades, obtém-se um número muito pequeno. Por isso, é possível haver dificuldades com a precisão dos pontos, como em casos de under-runs. Para evitar esse problema, é possível trabalhar com o logaritmo das probabilidades, de modo a aumentar os valores comparados.

* <b>Classificação dos tweets em duas categorias por vez.</b> Ao invés de utilizar diversas categorias, tipicamente, o método "uma em oposição às demais" costuma ser mais eficaz para o Naive Bayes. Em outras palavras, uma possível melhoria seria classificar primeiro em uma classe "A" e outra "todo resto", depois classe "B" e "todo resto" e assim por diante. 

* <b>Pré-processamento e análise dos dados.</b> Além da limpeza de pontuação, espeços, entre outros elementos, é possível realizar outros métodos de pré-processamento dos dados que podem auxiliar na eficácia do classificador Naive Bayes. A seguir, apresento alguns deles:

    <b>a)</b> Usar uma bilbioteca de stemming, como Porter, Lancaster e Snowball, para somar a contabilização de palavras com a mesma raiz. No código aqui demonstrado, por exemplo, as palavras "gostamos", "gostaria", "gostava" são contabilizadas singularmente. Utilizando essa ferramenta, poderia ser contabilizado a soma de suas freequências absolutas, como todas as palavras com raiz "gostar".
    
    <b>b)</b> Encontrar sinônimos. Da mesma forma que o stemming, gostar, apreciar, curtir, poderiam ser contabilizados como um único termo. 


___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**